In [1]:
from pathlib import Path
import pandas as pda
import sys
sys.path.append('../')
from src.DataHandling import DataHandling
%load_ext autoreload
%autoreload 2


In [2]:
project_folder_name = "../data/"
project_parameter_file = "parameter.yml"

dataHandler = DataHandling(project_folder_name, project_parameter_file)

In [3]:
raw_pdCommits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdCommits.p")
raw_pdUsers = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdUsers.p")
raw_pdReleases = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdReleases.p")
raw_pdEdits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdEdits.p")
raw_pdIssues = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdIssues.p")
raw_pdBranches = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdBranches.p")
raw_pdPullRequests = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdPullRequests.p")

## Check user list

In [4]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow"]
github_users_black_list_ids = raw_pdUsers [raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 
github_users_black_list_ids

['can-expect-late-month', 'would-hot-power-president']

In [5]:
raw_pdCommits['commiter_role'] = "student"
raw_pdCommits.loc[raw_pdCommits.author.isin(github_users_black_list_ids), "commiter_role"] = 'github'

In [6]:
raw_pdUsers[~raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids)]

,anonym_uuid,id,login,name,team
0,turn-democratic-month-house,MDQ6VXNlcjgyMDI5OTIz,Carl-Schade,None,camo
2,start-ready-open-hour,MDQ6VXNlcjgyMDE5OTI0,mokekei,None,camo
5,include-sit-best-father,MDQ6VXNlcjgyMTY0ODQ4,exe16,None,jado
7,use-can-red-family,MDQ6VXNlcjgyMDE2MDY5,DominikEbneter,None,jado
10,walk-wait-only-week,MDQ6VXNlcjgyMTcyMTc1,TheoPfannenschmidt,None,lote
11,tell-will-old-hand,MDQ6VXNlcjgyNzYxNjk2,lkunze304,None,lote
15,turn-serve-current-hand,MDQ6VXNlcjgyMzc3MzY2,MaxWilhelm65734,None,roma


## Eliminate none authors

In [7]:
raw_pdCommits[raw_pdCommits.author.isnull()]

,commit_sha,commited_at,no_of_modifications,commit_message,parent_sha,in_main_branch,author,committer,tag,branch_ids,team,task,commiter_role


In [8]:
missing_authors = {'robert.mende@chemie.tu-freiberg.de': 'mean-grow-real-child',
                   'RaginRain': "look-spend-dead-side",
                   'Divino': 'bring-political-fact-time'}

missing_authors_names = []

for index, row in raw_pdCommits[raw_pdCommits.author.isnull()].iterrows():
    print(row.author, ", ", row.unknown_user, " --> ", missing_authors[row.unknown_user])
    raw_pdCommits.at[index, "author"]  =  missing_authors[row.unknown_user]
    raw_pdCommits.at[index, "committer"]  =  missing_authors[row.unknown_user]

## Anonymize authors and teams

In [9]:
authors = raw_pdCommits.author.unique()
authors_dict = dict(zip(authors, range(len(authors))))

teams = raw_pdCommits.team.unique()
teams_dict = dict(zip(teams, range(len(teams))))

In [10]:
def annoymizeDataFrame(raw_df, teams_dict, authors_dict):
    df = raw_df.copy()
    if 'author' in df.columns:
        df['authorKey'] = df['author'].apply(lambda s: authors_dict.get(s) if s in authors_dict else s)
    if 'team' in df.columns:
        df['teamKey'] = df['team'].apply(lambda s: teams_dict.get(s) if s in teams_dict else s)
    return df

anonym_pdCommits = annoymizeDataFrame(raw_pdCommits, teams_dict, authors_dict)
anonym_pdReleases = annoymizeDataFrame(raw_pdReleases, teams_dict, authors_dict)
anonym_pdEdits = annoymizeDataFrame(raw_pdEdits, teams_dict, authors_dict)
anonym_pdIssues = annoymizeDataFrame(raw_pdIssues, teams_dict, authors_dict)
anonym_pdBranches = annoymizeDataFrame(raw_pdBranches, teams_dict, authors_dict)
anonym_pdPullRequests = annoymizeDataFrame(raw_pdPullRequests, teams_dict, authors_dict)

## Save files

In [11]:
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdCommits, "anonym_pdCommits.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdReleases, "anonym_pdReleases.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdEdits, "anonym_pdEdits.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdIssues, "anonym_pdIssues.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdBranches, "anonym_pdBranches.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdPullRequests, "anonym_pdPullRequests.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(teams_dict, "teams_dict.p")